# Test 6: Missing Transverse Energy Vector
Test my MET function by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class
Skip:  
2-1. Check the number of events for each branch  
2-2. Define mass quantities

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels
Skip

## 4. Jet Clustering

### 4-1. Select stable final state particles without/with filtering out dark sector

In [3]:
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [4]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v3 = myAnal_v3.jetClustering_v1(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering_v1(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done new version


## 5. Analyze the Jet in the Truth Level

### 5-1. Preselection version 1

### 5-2. MET
\begin{align}
    \vec{E}_T^\mathrm{miss} = -\sum_i {\vec{p}_T(i)}
\end{align}

In [5]:
arr_MET, df_MET = myAnal_v3.MET_visParticles_v1(SFSP_filterDM_v3)

19373 events in MET data.


In [6]:
arr_MET

array([(180.36140062,  2.80660527, -170.33591449,  59.29511843),
       (158.13608629, -0.31722873,  150.24565215, -49.32814406),
       (201.62627625, -0.37612596,  187.53151556, -74.06136607), ...,
       (116.27951432, -2.64619197, -102.30019373, -55.27744399),
       ( 73.07751061,  2.51891286,  -59.36221253,  42.61983435),
       (517.76480468,  1.51936352,   26.61835554, 517.08012543)],
      dtype=[('MET', '<f8'), ('phi', '<f8'), ('METx', '<f8'), ('METy', '<f8')])

In [7]:
df_MET

,MET,phi,METx,METy
0,180.361401,2.806605,-170.335914,59.295118
1,158.136086,-0.317229,150.245652,-49.328144
2,201.626276,-0.376126,187.531516,-74.061366
3,105.306424,2.742176,-97.017542,40.951673
4,190.593923,0.522398,165.173503,95.098673
...,...,...,...,...
19368,351.238421,-2.323326,-240.067150,-256.390702
19369,248.055171,-0.248239,240.451417,-60.946568
19370,116.279514,-2.646192,-102.300194,-55.277444
19371,73.077511,2.518913,-59.362213,42.619834


## 6. Check

### 6-1. Simple check

In [8]:
MET_v2 = myaFun_v2.parseMETTruthJet(SFSP_filterDM_v3, PseudoJet_filterDM_v3)

There are 19373 invisible events.
There are 19373 jet events.


In [9]:
print(len(MET_v2))
print(MET_v2[0].shape, MET_v2[1].shape)
print('-'*80)
print(np.sum(arr_MET['MET'] - MET_v2[0]))
print(np.sum(arr_MET['phi'] - MET_v2[1]))
(arr_MET['phi'] - MET_v2[1])[:5]

5
(19373,) (19373,)
--------------------------------------------------------------------------------
0.0
-61261.05674500097


array([ 0.        , -6.28318531, -6.28318531,  0.        ,  0.        ])

In [10]:
diff_phi = arr_MET['phi'] - MET_v2[1]
diff_phi2 = diff_phi[diff_phi > 0] - 2*np.pi
diff_phi3 = diff_phi2[diff_phi2 < 0] + 2*np.pi
np.sum(diff_phi3)

0.0

##### Conclusion: $\vec{E}_T^\mathrm{miss}$ of `MET_visParticle_v1` are the same as old version. Just $\phi$ is shifted $\pm \pi$.

## 7. Test

In [12]:
print(len(SFSP_filterDM_v3))
print(len(SFSP_filterDM_v3[0]))
print(len(SFSP_filterDM_v3[0][0]), len(SFSP_filterDM_v3[0][1]),
      len(SFSP_filterDM_v3[0][2]), len(SFSP_filterDM_v3[0][3]),
      len(SFSP_filterDM_v3[0][4]))
print(SFSP_filterDM_v3[0][0][:5])
SFSP_filterDM_v3[0][1][:5]

19373
5
169 169 169 169 169
[0.24688363 0.06091056 0.8968184  0.60751599 0.61821955]


array([3.89336681, 7.16352701, 1.30381167, 2.32303762, 1.83510363])